In [1]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime

## Group Assignment
### Team Number: 03
### Team Member Names: Sherry, Kelly, Ifan
### Team Strategy Chosen: SAFE

In [ ]:
tickers_df = pd.read_csv('Tickers_Example.csv')
#When the CSV file is read, it takes the first ticker and makes it the name of the column, so we need to extract it
tickers_list = list(tickers_df.columns)
#Now add the rest of the tickers in after it:
tickers_list += tickers_df.iloc[:, 0].tolist()
tickers_num = len(tickers_list)

#Dates to pull closing data from, change as needed
start_date = '2020-01-01'
end_date = '2022-01-02'

# Dictionary of all the stocks
stocks_dict = {}
stocks_full_info = {}

# Function produces the daily closing prices of a stock
def get_closing_prices(ticker, start, end):
    stock = yf.Ticker(ticker)
    hist = stock.history(start=start, end=end) #Daily
    return hist.Close

for i in range (tickers_num):
    stocks_dict[tickers_list[i]] = get_closing_prices(tickers_list[i], start_date, end_date)
    stocks_full_info[tickers_list[i]] = yf.Ticker(tickers_list[i])

# Create DataFrame
all_prices = pd.DataFrame(stocks_dict)

#Some tickers are delisted, so we drop those columns entirely
for i in range (tickers_num):
    if all_prices[tickers_list[i]].isnull().all():
        all_prices.drop(columns=[tickers_list[i]], inplace=True)

#Drop the rows at the bottom of the DataFrame where only a few stocks have values and where the dates are strange
all_prices = all_prices.dropna() 

#Display the entire DataFrame, so that we can manually check to make sure the values are sensible
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(all_prices)

- AGN: No data found, symbol may be delisted
- CELG: No data found, symbol may be delisted
- PCLN: No data found for this date range, symbol may be delisted
- RTN: No data found, symbol may be delisted


In [3]:
## doesnt work yet 
# Remove invalid stocks per assignment requirement (ones outside of the U.S & ones with average monthly volume of less than 200,000 shares)

def filter_market(df):
    df_transposed = df.T
    for i in range(len(df_transposed)):
        # Add country of market as a column to dataframe
        df_transposed["Market"].iloc[i] = stocks_full_info[i].info['market']
    
    # Drops the stocks in markets that are not in the U.S
    df_transposed.drop(df_transposed[df_transposed.Market != "us_market"].index, inplace=True)
    df_transposed = df_transposed.drop('Market', axis=1)
    return_df = df.T

    return return_df


def filter_volume(df):

    start = "2022-01-01"
    end = "2022-10-31"

    df_transposed = df.T

    for i in range(len(df_transposed)):
        ticker = yf.Ticker(df_transposed.iloc[i])
        ticker_hist = ticker.history(start=start, end=end)

        volume_hist = ticker_hist.loc[(ticker_hist.index >= pd.to_datetime(start)) & (ticker_hist.index <= pd.to_datetime(end))]
        df_transposed["Volume"].iloc[i] = (volume_hist.info['averageDailyVolume10Day'].mean())*3

    # Drops the stocks with volumes less than average monthly volume of 200,000
    df_transposed.drop(df_transposed[df_transposed.Volume < 200000].index, inplace=True)
    df_transposed = df_transposed.drop('Volume', axis=1)
    return_df = df.T

    return return_df

all_prices = all_prices.loc[filter_market(all_prices)]
all_prices = all_prices.loc[filter_volume(all_prices)]

# Drops stocks with more than specified number of NaN values
max_number_of_nans = 20
all_prices = all_prices.loc[:, (all_prices.isnull().sum(axis=0) <= max_number_of_nans)]


KeyError: 0

In [1]:
# Calculate BETA against S&P500
def stock_beta(df_column):

    index_ticker = yf.Ticker('SPY')
    index_prices = index_ticker.history(start=df_column.index[0], end=df_column.index[(len(df_column))])

    # Will need to double check this equation
    market_var= index_prices.var()
    beta = df_column.cov()/market_var
    
    return beta

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.